In [1]:
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup
from bs4 import Comment
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
import scipy.stats as st
import re
from openpyxl import load_workbook
import seaborn as sns
import timeit

## Database Schematic

<img src="database schematic4.png" alt="Drawing" style="width: 700px;"/>

In [2]:
import sqlite3
conn = sqlite3.connect('NBA_Fantasy_db.sqlite')
cur = conn.cursor()

cur.executescript('''
DROP TABLE IF EXISTS Game;
DROP TABLE IF EXISTS PlayerStats;
DROP TABLE IF EXISTS TeamStats
''')

cur.executescript('''
CREATE TABLE Game (
    id INTEGER NOT NULL PRIMARY KEY UNIQUE,
    date DATETIME,
    home_team_id INTEGER,
    road_team_id INTEGER,
    home_team_pts INTEGER,
    road_team_pts INTEGER,
    season_id INTEGER,
    day_of_week INTEGER,
    start_time VARCHAR(6),
    road_injured_players TEXT,
    home_injured_players TEXT
);
CREATE TABLE TeamStats (
    game_id INTEGER,
    team_id INTEGER,
    team_Pace FLOAT,
    team_eFGp FLOAT,
    team_TOVp FLOAT,
    team_ORBp FLOAT,
    team_FTvFGA FLOAT,
    team_ORtg FLOAT,
    PRIMARY KEY (game_id, team_id)
);
CREATE TABLE PlayerStats (
    game_id INTEGER,
    player_id INTEGER,
    team_id INTEGER,
    min FLOAT,
    FG INTEGER,
    FGA INTEGER,
    ThreeP INTEGER,
    ThreePA INTEGER,
    FT INTEGER,
    FTA INTEGER,
    ORB INTEGER,
    DRB INTEGER,
    AST INTEGER,
    STL INTEGER,
    BLK INTEGER,
    TOV INTEGER,
    PF INTEGER,
    PTS INTEGER,
    PlusMinus INTEGER,
    TSp FLOAT,
    eFGp FLOAT,
    ThreePAr FLOAT,
    FTr FLOAT,
    ORBp FLOAT,
    DRBp FLOAT,
    TRBp FLOAT,
    ASTp FLOAT,
    STLp FLOAT,
    BLKp FLOAT,
    TOVp FLOAT,
    USGp FLOAT,
    ORtg FLOAT,
    DRtg FLOAT,
    BPM FLOAT,  
    PRIMARY KEY (game_id, player_id)
)
''')
conn.commit()

## Create tables for game, player,  and team stats

In [3]:
def get_inactive_players(box_score_bsObj):
    """
    This function takes in a BeautifulSoup object and identifies the inactive players for each team.
    Each list of players is concatenated into a csv string and returned
    """
    home_inactive = []
    road_inactive = []
        
    team_injury_flag = 0
    for div in box_score_bsObj.find_all('div'):
        if re.match('^Inactive', div.get_text()):
            for inactive_player in div.children:
                try:
                    injured_player = inactive_player.get_text().strip()
                        
                    if len(injured_player) == 3 and team_injury_flag == 0: # injured_player == road team code
                        team_injury_flag = 1
                    elif len(injured_player) == 3 and team_injury_flag == 1: # injured_player == home team code
                        team_injury_flag = 2
                    elif team_injury_flag == 1: # len != 3 and flag == 1 so this is injured road player
                        road_inactive.append(injured_player)
                    elif team_injury_flag == 2: # len != 3 and flag == 2 so this is injured home player
                        home_inactive.append(injured_player)      
                except:
                    pass
    # convert names of injured players into their respective ids            
    for i, name in enumerate(home_inactive):
        name_SQL_like = name + '%' # add % for like statement 
        try:
            # use LIKE instead of == since the website has discrepencies with suffixes 
            cur.execute('''SELECT id FROM Player WHERE name LIKE ?''', (name_SQL_like,))
            home_inactive[i] = str(cur.fetchone()[0])
        except:
            home_inactive[i] = ''
            if name not in failed:
                failed.append(name)
    for i, name in enumerate(road_inactive):
        name_SQL_like = name + '%' # add % for like statement
        try:
            cur.execute('''SELECT id FROM Player WHERE name LIKE ?''', (name_SQL_like,))
            road_inactive[i] = str(cur.fetchone()[0])
        except:
            road_inactive[i] = ''
            if name not in failed:
                failed.append(name)
        
    # convert list of injured players into csv string 
    home_inactive = ','.join(home_inactive)
    road_inactive = ','.join(road_inactive)
    
    return home_inactive, road_inactive


In [4]:
def get_team_stats(box_score_bsObj, game_id, home_team_id, road_team_id):
    """
    This function retrieves the data from the table on each team's "four factors" (which actually has six factors)
    These data are then added the the TeamStats table
    """
    
    four_factors = box_score_bsObj.find('div', attrs={'id': ['all_four_factors']})
    for comment in four_factors(text=lambda text: isinstance(text, Comment)):
        comment = BeautifulSoup(comment, 'html.parser')
        road_team_stats = []
        home_team_stats = []
        flag=0
        table = comment.find('tbody')
        for team in table.children:
            try:
                for stat in team.children:
                    if flag == 0:
                        road_team_stats.append(stat.get_text())
                    else:
                        home_team_stats.append(stat.get_text())
                flag = 1
            except:
                pass
    # first item in list is three digit team code - get rid of this since we already have it
    road_team_stats = road_team_stats[1:]
    home_team_stats = home_team_stats[1:]
    
    # everything in the list is a string but needs to be converted to float
    road_team_stats = [float(stat) for stat in road_team_stats]
    home_team_stats = [float(stat) for stat in home_team_stats]
    
    road_team_stats = tuple([game_id, road_team_id] + road_team_stats)
    home_team_stats = tuple([game_id, home_team_id] + home_team_stats)
    
    cur.execute('''
    INSERT INTO TeamStats (game_id, team_id, team_Pace, team_eFGp, team_TOVp, team_ORBp, team_FTvFGA, team_ORtg)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)''', road_team_stats)
    cur.execute('''
    INSERT INTO TeamStats (game_id, team_id, team_Pace, team_eFGp, team_TOVp, team_ORBp, team_FTvFGA, team_ORtg)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)''', home_team_stats)
    conn.commit()

In [5]:
def insert_stats_db(stats, game_id, team_id, basic=True):
    """
    This function takes a list of stats for a player and adds them to the PlayerStats table
    There are two types of data: basic and advanced, so each are processed differently
    """
    player_name = stats[0] + '%'
    stats = stats[1:] # get rid of player name
    cur.execute('SELECT id FROM Player WHERE name LIKE ?', (player_name,))
    try:
        player_id = cur.fetchone()[0]
    except:
        if player_name[:-1] not in failed:
            failed.append(player_name[:-1])
    
    if basic:
        # get rid of FG%, FT%, 3P%, and ORB since they are redundant
        del stats[12]
        del stats[9]
        del stats[6]
        del stats[3]
        
        # convert min to float
        time = stats[0]
        [minutes, seconds] = re.split(':', time)
        minutes = int(minutes)
        seconds = int(seconds)
        stats[0] = round(minutes + (seconds / 60), 1)
        
        stats = tuple([game_id, player_id, team_id] + stats)
        
        cur.execute('''
        INSERT INTO PlayerStats (game_id, player_id, team_id, min, FG, FGA, ThreeP, ThreePA, FT,
                                 FTA, ORB, DRB, AST, STL, BLK, TOV, PF, PTS, PlusMinus)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', stats)
    
    else:  
        stats = stats[1:] # get rid of minutes played
        stats_where = tuple(stats + [game_id, player_id])
        cur.execute('''
        UPDATE PlayerStats SET TSp = ?, eFGp = ?, ThreePAr = ?, FTr = ?, ORBp = ?, DRBp = ?, TRBp = ?, 
        ASTp = ?, STLp = ?, BLKp = ?, TOVp = ?, USGp = ?, ORtg = ?, DRtg = ?, BPM = ?
        WHERE game_id == ? AND player_id == ?''', stats_where)
        
    conn.commit()

In [6]:
def get_player_stats(box_score_bsObj, game_id, home_team, road_team, home_team_id, road_team_id):
    """
    This function takes a BeautifulSoup object for a particular game's box score
    Each box score has four tables of player stats
    Two tables per team - one for basic and the other for advanced stats
    The stats are collected from each row and passed to a function that adds them to the PlayerStats table
    """
    divs = box_score_bsObj.find_all('div', attrs={'class':'table_container'})
    caption = ''
    for div in divs:
        try:
            prev_caption = caption
            caption = div.caption.get_text()
            # One - road team / basic
            if re.search('{} \([0-9]'.format(road_team), caption):
                table = div.find('tbody')
                for child in table.children:
                    try:
                        stats = []
                        for stat in child.children:
                            stats.append(stat.get_text())
                        insert_stats_db(stats, game_id, road_team_id, basic=True)
                    except:
                        pass
            # Two - home team / basic
            if re.search('{} \([0-9]'.format(home_team), caption):
                table = div.find('tbody')
                for child in table.children:
                    try:
                        stats = []
                        for stat in child.children:
                            stats.append(stat.get_text())
                        insert_stats_db(stats, game_id, home_team_id, basic=True)
                    except:
                        pass
                    
            # Three - road team / advanced
            if re.search('^ Table$', caption) and (road_team in prev_caption):
                table = div.find('tbody')
                for child in table.children:
                    try:
                        stats = []
                        for stat in child.children:
                            stats.append(stat.get_text())
                        insert_stats_db(stats, game_id, road_team_id, basic=False)
                    except:
                        pass
                
            # Four - home team / advanced
            if re.search('^ Table$', caption) and (home_team in prev_caption):
                table = div.find('tbody')
                for child in table.children:
                    try:
                        stats = []
                        for stat in child.children:
                            stats.append(stat.get_text()) 
                        insert_stats_db(stats, game_id, home_team_id, basic=False)
                    except:
                        pass
        except:
            pass

In [7]:
def get_stats_month(url):
    """
    This function adds stats to Game, PlayerStats, and TeamStats tables in the database for a particular month
    """
    bsObj = BeautifulSoup(url.read())
    for game in bsObj.find('tbody').children:
        # get the date and day of week (use this as a check to make sure this is a valid game)
        try:
            date = game.find('th').get_text()
            date = pd.to_datetime(date)
            date = date.date()
            day_of_week = date.weekday()
        except:
            continue # need this to eliminate header rows interspersed among games
        
        # get start time and home/road teams
        row = game.find_all('td')
        start_time = row[0].get_text() # e.g. 7:30p
        road_team = row[1].get_text() # e.g. New York Knicks
        road_team_pts = row[2].get_text()
        home_team = row[3].get_text()
        home_team_pts = row[4].get_text()
        
        # get road/home team id's
        cur.execute('''SELECT id FROM Team WHERE name == ?''', (road_team,))
        road_team_id = cur.fetchone()[0]
        cur.execute('''SELECT id FROM Team WHERE name == ?''', (home_team,))
        home_team_id = cur.fetchone()[0]
        
        # get the link to the box score
        box_score_url = game.find_all('a')[3].get('href')
        box_score_url = urlopen('https://www.basketball-reference.com{}'.format(box_score_url))
        box_score_bsObj = BeautifulSoup(box_score_url.read())
        
        # get list of inactive players 
        home_inactive, road_inactive = get_inactive_players(box_score_bsObj)
        
        # add data into Game table in database
        cur.execute('''INSERT INTO GAME (date, home_team_id, road_team_id, home_team_pts, road_team_pts, 
                                         season_id, day_of_week, start_time,
                                         road_injured_players, home_injured_players)
                       VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', 
                    (date, home_team_id, road_team_id, home_team_pts, road_team_pts, season_id, day_of_week, 
                     start_time, road_inactive, home_inactive)
                   )
        conn.commit()
        
        # get id of newly added row in Game table
        cur.execute('''SELECT id FROM GAME WHERE date == ? AND home_team_id = ?''', (date, home_team_id))
        game_id = cur.fetchone()[0]
        
        # add player/team stats
        get_team_stats(box_score_bsObj, game_id, home_team_id, road_team_id)
        get_player_stats(box_score_bsObj, game_id, home_team, road_team, home_team_id, road_team_id)

## Main Code

In [8]:
start_time = timeit.time.time()
years = ['2017', '2018', '2019', '2020', '2021']
failed = []
for year in years:
    print('Starting year ', year)
    season_id = int(year) - 2016
    
    # get list of months for each season
    months = []   
    url = urlopen('https://www.basketball-reference.com/leagues/NBA_{}_games.html'.format(year))
    bsObj = BeautifulSoup(url.read())
    months_table = bsObj.find('div', attrs={'class': 'filter'})
    for month in months_table.children:
        try:
            months.append(month.get_text().strip().lower())
        except:
            pass
    
    for month in months:
        # account for pandemic causing there to be two Octobers in the 2020 season
        if re.search(' ', month):
            [pandemic_month, pandemic_year] = re.split(' ', month)
            url = urlopen('https://www.basketball-reference.com/leagues/NBA_{}_games-{}-{}.html'.format(
                year, pandemic_month, pandemic_year))
        else:
            url = urlopen('https://www.basketball-reference.com/leagues/NBA_{}_games-{}.html'.format(year, month))
        get_stats_month(url)
        
    print('This code has taken {:.2f} minutes so far'.format((timeit.time.time() - start_time)/60))
 
print('')
print('The following names were not in the Player table:')
print('')
print(failed)
# note that some players do not make it into the list of injuries because they were not on the rosters on
# basketball reference usually because they were retired or did not make the team after signing.
# Either way, these players are not going to significantly contribute to the injury feature

Starting year  2017
This code has taken 14.84 minutes so far
Starting year  2018
This code has taken 29.46 minutes so far
Starting year  2019
This code has taken 44.15 minutes so far
Starting year  2020
This code has taken 56.93 minutes so far
Starting year  2021
This code has taken 70.27 minutes so far

The following names were not in the Player table:

['Mo Williams', 'Festus Ezeli', 'Chris Bosh', 'Nikola Peković', 'Ray McCallum', 'Glenn Robinson III', 'Yakuba Ouattara', 'Michael Young', 'Carrick Felix', 'Eric Griffin', 'Jaylen Johnson', "Le'Bryan Nash", 'Keenan Evans', 'Mohamed Bamba', 'Billy Preston', 'Trevon Bluiett', 'Stephan Hicks', 'Richard Solomon', 'Justin Bibbs', 'Cody Demps', 'Tarik Phillip', 'Robert Franks', 'Ignas Brazdeikis', 'Levi Randolph', 'Kenny Wooten', "Sir'Dominic Pointer", 'Frank Mason III', 'Karim Mané', 'Dakota Mathias', 'Will Magnay', 'Ashton Hagans', 'Greg Whittington', 'Mason Jones', 'Devin Cannady', 'Malik Fitts', 'Myles Powell', 'Marcos Louzada Silva', 'Ca

In [9]:
conn.close()